After you have done step1SaveMtxToPt.cpp, you should get

heyuhao@sutd:~/AMMBench$ ls /home/heyuhao/AMMBench/build/benchmark/torchscripts/VQ/AMME2E/MtxPt

AST_A.pt  AST_B.pt  BUS_A.pt  BUS_B.pt  DWAVE_A.pt  DWAVE_B.pt  ECO_A.pt  ECO_B.pt  QCD_A.pt  QCD_B.pt  RDB_A.pt  RDB_B.pt  UTM_A.pt  UTM_B.pt  ZENIOS_A.pt  ZENIOS_B.pt

This jupuyer is to generate the codeword & lookup table for each pair of matrices

In [12]:
import torch
from sklearn.cluster import KMeans
from torch import nn

In [13]:
def getCodewordAndLookUpTable(A, B, m):
    
    # Sample matrix sizes and PQ parameters
    A_rows, A_cols = A.shape
    lA = A_rows // m // 10  # Number of centroids for each subspace
    CA = A_cols // m  # Dimension of each subspace

    # Sample matrix sizes and PQ parameters
    B_rows, B_cols = B.shape
    lB = B_cols // m // 10  # Number of centroids for each subspace
    CB = B_rows // m  # Dimension of each subspace

    # Initialize lists to store subspaces and centroids
    subspaces_A = []
    codewords_A = []

    # Create subspaces and centroids
    for i in range(m):
        subspace_A = A[:, i * CA : (i + 1) * CA]  # 500*20
        subspaces_A.append(subspace_A)

        # Apply KMeans on the row vectors within the subspace
        kmeans = KMeans(n_clusters=lA, n_init=10)
        kmeans.fit(subspace_A)  # 500*20
        subspace_centroids_A = torch.tensor(kmeans.cluster_centers_) # 10*20
        codewords_A.append(subspace_centroids_A)

    # Convert lists to tensors
    subspaces_A = torch.stack(subspaces_A, dim=0)
    codewords_A = torch.stack(codewords_A, dim=0) # 5*10*20

    print("Subspaces A shape:", subspaces_A.shape) # torch.Size([5, 500, 20])
    print("Codewords A shape:", codewords_A.shape) # torch.Size([5, 10, 20])


    # Initialize lists to store subspaces and centroids
    subspaces_B = []
    codewords_B = []

    # Create subspaces and centroids
    for k in range(m):
        subspace_B = B[k * CB : (k + 1) * CB, :]  # Extract the subspace along x-axis
        subspaces_B.append(subspace_B)

        # Apply KMeans on the column vectors within the subspace
        kmeans = KMeans(n_clusters=lB, n_init=10)
        kmeans.fit(subspace_B.T)  # Transpose to cluster along columns (column vectors)
        subspace_centroids_B = torch.tensor(kmeans.cluster_centers_)
        codewords_B.append(subspace_centroids_B)

    # Convert lists to tensors
    subspaces_B = torch.stack(subspaces_B, dim=0)
    codewords_B = torch.stack(codewords_B, dim=0)

    print("Subspaces B shape:", subspaces_B.shape)  # torch.Size([5, 20, 300])
    print("Codewords B shape:", codewords_B.shape)  # torch.Size([5, 6, 20])

    # Sample precomputed codewords for A and B (You should replace these with your actual codewords)
    lookup_table = torch.zeros((m,lA,lB))
    for i in range(m):
        for j in range(lA):
            for k in range(lB):
                lookup_table[i][j][k] = torch.matmul(codewords_A[i][j], codewords_B[i][k]) # for each subspace, get catersian product of A,B codeword

    print("lookup_table.shape: ", lookup_table.shape)

    return lA, lB, codewords_A, codewords_B, lookup_table

In [14]:
# used to save codeword and lookup_table to pt
class TensorContainer(nn.Module):
    def __init__(self, tensor_dict):
        super().__init__()
        for key,value in tensor_dict.items():
            setattr(self, key, value)


In [15]:
def quantize(C, D, codewords_A, codewords_B, lookup_table):
    m = codewords_A.shape[0]

    # Sample matrix sizes and PQ parameters
    C_rows, C_cols = C.shape
    CC = codewords_A.shape[2]  # Dimension of each subspace for matrix C

    D_rows, D_cols = D.shape
    CD = codewords_B.shape[2]  # Dimension of each subspace for matrix D

    # Initialize lists to store quantized indices
    C_quantized = []
    D_quantized = []

    # Find the nearest codeword indices for matrix C
    for i in range(m):
        codewords_c = codewords_A[i] # torch.Size([10, 20])
        C_subspace = C[:, i * CC : (i + 1) * CC] # torch.Size([500, 20])
        distances = torch.norm(codewords_c.unsqueeze(0) - C_subspace.unsqueeze(1), dim=2, p=2) # torch.Size([500, 10]) = torch.Size([1, 10, 20]) - torch.Size([500, 1, 20])
        closest_codeword_indices = torch.argmin(distances, dim=1) # torch.Size([500])
        C_quantized.append(closest_codeword_indices)
    C_quantized = torch.stack(C_quantized, dim=1) # torch.Size([500, 5])
    # print("C quantized shape:", C_quantized.shape)  # Shape of the quantized indices for C

    # Find the nearest codeword indices for matrix D
    for k in range(m):
        codewords_d = codewords_B[k]
        D_subspace = D[k * CD : (k + 1) * CD, :] # torch.Size([20, 300])
        distances = torch.norm(codewords_d.unsqueeze(0) - torch.swapaxes(D_subspace.unsqueeze(1), 0, 2), dim=2, p=2) # torch.Size([300, 6]) = torch.Size([1, 6, 20]) - torch.Size([300, 1, 20])
        closest_codeword_indices = torch.argmin(distances, dim=1) # torch.Size([300])
        D_quantized.append(closest_codeword_indices.T)
    D_quantized = torch.stack(D_quantized, dim=0) # torch.Size([5, 300])
    # print("D quantized shape:", D_quantized.shape)  # Shape of the quantized indices for D

    # Define the batch size for batch processing
    batch_size_C = C_rows
    batch_size_D = D_cols

    # Initialize the matrix products
    matrix_products = torch.zeros((C_rows, D_cols))

    # Perform matrix multiplication using batch processing
    for i in range(0, C_rows, batch_size_C):
        for j in range(0, D_cols, batch_size_D):
            batch_result = torch.zeros((batch_size_C, batch_size_D))
            
            for k in range(m):
                # Gather quantized indices for the current batch
                C_indices = C_quantized[i:i+batch_size_C, k]
                D_indices = D_quantized[k, j:j+batch_size_D]
                
                # Gather relevant entries from the lookup table
                batch_lookup = lookup_table[k, C_indices, :][:, D_indices]
                
                # Accumulate the batch result
                batch_result += batch_lookup
            
            # Assign the batch result to the corresponding position in the matrix products
            matrix_products[i:i+batch_size_C, j:j+batch_size_D] = batch_result

    E = torch.matmul(C, D)
    return torch.norm(matrix_products-E)/torch.norm(E)

In [16]:
datasetDir = '/home/heyuhao/AMMBench/build/benchmark/torchscripts/VQ/AMME2E/MtxPt'
saveDir = '/home/heyuhao/AMMBench/benchmark/torchscripts/VQ/AMME2E/CodewordLookUpTable'

for datasetName in ["AST","BUS","DWAVE","ECO","QCD","RDB","UTM","ZENIOS"]:
    for m in [1, 10]: # Number of subspaces

        # load
        A = torch.load(f'{datasetDir}/{datasetName}_A.pt')
        B = torch.load(f'{datasetDir}/{datasetName}_B.pt')

        # calculate codeword and lookup_table
        lA, lB, codewords_A, codewords_B, lookup_table = getCodewordAndLookUpTable(A, B, m)

        # calculate error
        C=A # actually C,D are testing matrices, and usually different from training matrices A,B. but its ok to make them the same also, cuz we more focus on latency
        D=B
        relativeFroError = quantize(C, D, codewords_A, codewords_B, lookup_table)

        # save codeword and lookup_table
        tensor_dict = {
            # 'A': A,
            # 'B': B,
            'codewordsA': codewords_A,
            'codewordsB': codewords_B,
            'lookUpTable': lookup_table,
            'datasetName': datasetName,
            'm': m,
            'lA': lA,
            'lB': lB,
            'relativeFroError': relativeFroError
        }
        tensors = TensorContainer(tensor_dict)
        tensors = torch.jit.script(tensors)
        tensors.save(f'{saveDir}/{datasetName}_m{m}_lA{lA}_lB{lB}.pth')
        print(datasetName, m, lA, lB, relativeFroError)

Subspaces A shape: torch.Size([1, 765, 765])
Codewords A shape: torch.Size([1, 76, 765])
Subspaces B shape: torch.Size([1, 765, 765])
Codewords B shape: torch.Size([1, 76, 765])
lookup_table.shape:  torch.Size([1, 76, 76])
AST 1 76 76 tensor(1.2885e-07)


/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Subspaces A shape: torch.Size([10, 765, 76])
Codewords A shape: torch.Size([10, 7, 76])


/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Subspaces B shape: torch.Size([10, 76, 765])
Codewords B shape: torch.Size([10, 7, 76])
lookup_table.shape:  torch.Size([10, 7, 7])
AST 10 7 7 tensor(0.0065)
Subspaces A shape: torch.Size([1, 4929, 10595])
Codewords A shape: torch.Size([1, 492, 10595])
Subspaces B shape: torch.Size([1, 10595, 4929])
Codewords B shape: torch.Size([1, 492, 10595])
lookup_table.shape:  torch.Size([1, 492, 492])
BUS 1 492 492 tensor(5.7282e-06)
Subspaces A shape: torch.Size([10, 4929, 1059])
Codewords A shape: torch.Size([10, 49, 1059])
Subspaces B shape: torch.Size([10, 1059, 4929])
Codewords B shape: torch.Size([10, 49, 1059])
lookup_table.shape:  torch.Size([10, 49, 49])
BUS 10 49 49 tensor(9.7871e-05)
Subspaces A shape: torch.Size([1, 512, 512])
Codewords A shape: torch.Size([1, 51, 512])
Subspaces B shape: torch.Size([1, 512, 512])
Codewords B shape: torch.Size([1, 51, 512])
lookup_table.shape:  torch.Size([1, 51, 51])
DWAVE 1 51 51 tensor(0.0019)
Subspaces A shape: torch.Size([10, 512, 51])
Codewords

...

Subspaces B shape: torch.Size([1, 2873, 2873])
Codewords B shape: torch.Size([1, 287, 2873])
lookup_table.shape:  torch.Size([1, 287, 287])
ZENIOS 1 287 287 tensor(1.2641e-08)


/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuha

Subspaces A shape: torch.Size([10, 2873, 287])
Codewords A shape: torch.Size([10, 28, 287])


/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuhao/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (28). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/heyuha

Subspaces B shape: torch.Size([10, 287, 2873])
Codewords B shape: torch.Size([10, 28, 287])
lookup_table.shape:  torch.Size([10, 28, 28])
ZENIOS 10 28 28 tensor(0.0010)


In [17]:
import numpy as np

# Assuming matrix_products is your 2D tensor
max_value = torch.max(matrix_products).item()
min_value = torch.min(matrix_products).item()
mean_value = torch.mean(matrix_products).item()

# Convert tensor to a flattened 1D tensor
flattened_matrix = matrix_products.view(-1)

# Convert the tensor to a NumPy array for computing percentiles
np_matrix = flattened_matrix.numpy()

percentile_25 = np.percentile(np_matrix, 25)
percentile_75 = np.percentile(np_matrix, 75)

print("Maximum:", max_value)
print("Minimum:", min_value)
print("Mean:", mean_value)
print("25th Percentile:", percentile_25)
print("75th Percentile:", percentile_75)


NameError: name 'matrix_products' is not defined